<a href="https://colab.research.google.com/github/chanyelee/Dacon-electricity-forecast/blob/main/SAINT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.4 scipy==1.11.4
!pip install -q pytorch-widedeep

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')

try:
    path = '/content/drive/MyDrive/위아이티'
    os.chdir(path)
    print(f"현재 작업 디렉터리: {os.getcwd()}")
except FileNotFoundError:
    print(f"오류: '{path}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")

print("\n현재 폴더 내 파일 목록:")
!ls

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=ResourceWarning)

In [ ]:
import numpy as np
import pandas as pd
import holidays
import math
import copy
import joblib

In [ ]:
import torch
import torch.nn as nn
from pytorch_widedeep.models import WideDeep, SAINT
from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.training import Trainer
from pytorch_widedeep.initializers import XavierNormal
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_widedeep.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# 1. GPU 장치 설정
# torch.cuda.is_available()는 CUDA를 지원하는 GPU가 있는지 확인하는 함수입니다.
# GPU가 있으면 'cuda'로, 없으면 'cpu'로 장치를 자동으로 설정합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. 설정된 장치 확인
print(f"✅ Using device: {device}")

# 3. GPU 사용 시, 이름과 메모리 상태 출력 (선택 사항이지만 확인에 유용)
if device.type == 'cuda':
    print(f"✅ GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**3:.1f} GB")
    print(f"Memory Cached:    {torch.cuda.memory_reserved(0)/1024**3:.1f} GB")

✅ Using device: cpu


In [ ]:
train_group_1 = pd.read_csv('train_group_1.csv')
train_group_2 = pd.read_csv('train_group_2.csv')
train_group_3 = pd.read_csv('train_group_3.csv')

valid_group_1 = pd.read_csv('valid_group_1.csv')
valid_group_2 = pd.read_csv('valid_group_2.csv')
valid_group_3 = pd.read_csv('valid_group_3.csv')

test_group_1 = pd.read_csv('test_group_1.csv')
test_group_2 = pd.read_csv('test_group_2.csv')
test_group_3 = pd.read_csv('test_group_3.csv')

In [ ]:
def optimize_memory(df: pd.DataFrame) -> pd.DataFrame:
    """
    데이터프레임의 int64, float64 타입을 각각 int32, float32로 변경하여 메모리를 최적화합니다.
    """

    # 변경 전 메모리 사용량 확인
    mem_before = df.memory_usage(deep=True).sum()
    print(f"--- 변경 전 메모리: {mem_before / 1e6:.2f} MB ---")

    # 변경할 타입을 담을 딕셔너리 생성
    conversion_dict = {}
    for col, dtype in df.dtypes.items():
        if dtype == 'int64':
            conversion_dict[col] = 'int32'
        elif dtype == 'float64':
            conversion_dict[col] = 'float32'

    # astype을 이용해 한 번에 변경
    if conversion_dict:
        df = df.astype(conversion_dict)
        mem_after = df.memory_usage(deep=True).sum()

        print(f"--- 변경 후 메모리: {mem_after / 1e6:.2f} MB ---")

        # 메모리 절감량 요약
        reduction = ((mem_before - mem_after) / mem_before) * 100
        print(f"✅ 총 {reduction:.2f}%의 메모리를 절감했습니다.")
    else:
        print("✅ 변경할 int64 또는 float64 타입의 컬럼이 없습니다.")

    return df

In [ ]:
train_group_1 = optimize_memory(train_group_1)
train_group_2 = optimize_memory(train_group_2)
train_group_3 = optimize_memory(train_group_3)

valid_group_1 = optimize_memory(valid_group_1)
valid_group_2 = optimize_memory(valid_group_2)
valid_group_3 = optimize_memory(valid_group_3)

test_group_1 = optimize_memory(test_group_1)
test_group_2 = optimize_memory(test_group_2)
test_group_3 = optimize_memory(test_group_3)

--- 변경 전 메모리: 21.70 MB ---
--- 변경 후 메모리: 13.69 MB ---
✅ 총 36.91%의 메모리를 절감했습니다.
--- 변경 전 메모리: 19.49 MB ---
--- 변경 후 메모리: 12.22 MB ---
✅ 총 37.29%의 메모리를 절감했습니다.
--- 변경 전 메모리: 6.03 MB ---
--- 변경 후 메모리: 3.74 MB ---
✅ 총 37.98%의 메모리를 절감했습니다.
--- 변경 전 메모리: 2.14 MB ---
--- 변경 후 메모리: 1.35 MB ---
✅ 총 36.91%의 메모리를 절감했습니다.
--- 변경 전 메모리: 1.92 MB ---
--- 변경 후 메모리: 1.21 MB ---
✅ 총 37.28%의 메모리를 절감했습니다.
--- 변경 전 메모리: 0.59 MB ---
--- 변경 후 메모리: 0.37 MB ---
✅ 총 37.97%의 메모리를 절감했습니다.
--- 변경 전 메모리: 2.01 MB ---
--- 변경 후 메모리: 1.29 MB ---
✅ 총 36.09%의 메모리를 절감했습니다.
--- 변경 전 메모리: 1.81 MB ---
--- 변경 후 메모리: 1.15 MB ---
✅ 총 36.51%의 메모리를 절감했습니다.
--- 변경 전 메모리: 0.56 MB ---
--- 변경 후 메모리: 0.35 MB ---
✅ 총 37.28%의 메모리를 절감했습니다.


In [ ]:
TARGET_COL = '전력소비량(kWh)'
TARGET_COL_LOG = '전력소비량(kWh)_log'

CAT_COLS = ['건물번호', 'hour', '요일구분', '월', '요일', '건물유형', '강수여부']

g1_cont_cols = [
    '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)_log', '전력소비량(kWh)', 'hour_sin', 'hour_cos', 'Time_Index',
    '연면적(m2)', '냉방면적(m2)', '일조(hr)', '일사(MJ/m2)', '체감온도', '온습도지수', '일주일_전_전력소비량',
    '1시간_전_전력소비량', '하루_전_전력소비량',
]

g2_cont_cols = g1_cont_cols + ['태양광용량(kW)']
g3_cont_cols = g2_cont_cols + ['ESS저장용량(kWh)', 'PCS용량(kW)']

In [ ]:
def pretrain_saint_model(
    pretrain_dfs: list,
    cat_cols: list,
    cont_cols: list,
    pretrain_epochs: int = 10
):
    """
    '가짜 Target' 방식을 사용하여 안정적으로 자기지도학습을 수행하는 함수
    """
    print("===== 🚀 1. SAINT 자기지도학습 시작 =====")

    all_features_df = pd.concat(pretrain_dfs, ignore_index=True, sort=False)

    preprocessor = TabPreprocessor(
        cat_embed_cols=cat_cols,
        continuous_cols=cont_cols,
        scaler="minmax",
        impute_missing=True
    )
    preprocessor.fit(all_features_df)

    model = WideDeep(deeptabular=SAINT(
        column_idx=preprocessor.column_idx,
        cat_embed_input=preprocessor.cat_embed_input,
        continuous_cols=cont_cols
    ))
    initializers = XavierNormal()

    pretrain_optimizer = AdamW(model.parameters(), lr=5e-4, eps=1e-7)
    trainer_pretrain = Trainer(
        model=model,
        objective='regression',
        optimizers=pretrain_optimizer,
        initializers=initializers,
        clip_grad_norm=1.0
    )

    X_pretrain = preprocessor.transform(all_features_df).astype('float32')

    dummy_target = np.zeros((len(X_pretrain), 1))

    trainer_pretrain.fit(
        X_tab=X_pretrain,
        target=dummy_target, # 가짜 타겟 전달
        pretrain_method='masked',
        val_split=0.1, # 가짜 타겟이 있으므로 val_split 사용
        n_epochs=pretrain_epochs,
        batch_size=512
    )

    print("✅ 자기지도학습 완료")
    return model, preprocessor

In [ ]:
def finetune_saint_model(
    model,
    preprocessor,
    finetune_df: pd.DataFrame,
    target_col: str,
    finetune_epochs: int = 100
):
    """
    사전학습된 SAINT 모델을 특정 데이터셋으로 미세조정(Fine-tuning)합니다.
    """
    print(f"===== 🚀 2. {target_col}에 대한 미세조정 시작 =====")

    X_finetune = preprocessor.transform(finetune_df).astype('float32')
    y_finetune = finetune_df[target_col].values.reshape(-1, 1)

    y_scaler = StandardScaler()
    y_scaled = y_scaler.fit_transform(y_finetune)

    finetune_optimizer = AdamW(model.parameters(), lr=1e-4, eps=1e-7)
    finetune_scheduler = ReduceLROnPlateau(finetune_optimizer, mode='min', factor=0.2, patience=5)
    callbacks = [EarlyStopping(patience=15, monitor='val_loss')]

    trainer_finetune = Trainer(
        model=model,
        objective='mae',
        optimizers=finetune_optimizer,
        lr_schedulers=finetune_scheduler,
        callbacks=callbacks,
        clip_grad_norm=1.0
    )

    trainer_finetune.fit(
        X_tab=X_finetune,
        target=y_scaled,
        n_epochs=finetune_epochs,
        batch_size=64,
        val_split=0.2
    )

    print("✅ 미세조정 완료")
    return model, y_scaler

In [ ]:
def run_full_training_pipeline(
    pretrain_dfs: list,
    finetune_groups: dict,
    target_col: str, # 로그 변환된 타겟 컬럼 이름 (예: '전력소비량(kWh)_log')
    cat_cols: list,
    common_cont_cols: list,
    pretrain_epochs: int = 10,
    finetune_epochs: int = 100
):
    """
    자가학습부터 그룹별 미세조정까지 전체 파이프라인을 실행합니다.
    전처리기는 피처(feature)만으로 학습됩니다.
    """
    # --- 타겟 컬럼 이름 정의 ---
    # 로그 타겟 이름으로부터 원본 타겟 이름을 추정 (예: '_log' 제거)
    original_target_col = target_col.replace('_log', '')

    # 1. 사전학습용 데이터(DataFrame)에서 타겟 컬럼들 제거
    pretrain_features_dfs = []
    for df in pretrain_dfs:
        features_only_df = df.drop(columns=[target_col, original_target_col], errors='ignore')
        pretrain_features_dfs.append(features_only_df)

    # 2. 사전학습용 컬럼 이름 리스트(list)에서 타겟 컬럼들 제거
    cont_cols_for_pretrain = [
        col for col in common_cont_cols
        if col not in [target_col, original_target_col]
    ]

    # 3. 자가학습(Pre-training) 수행
    # 이제 데이터와 컬럼 리스트 모두 타겟 정보가 없는 상태로 전달됩니다.
    base_model, preprocessor = pretrain_saint_model(
        pretrain_dfs=pretrain_features_dfs,          # 타겟이 제거된 데이터
        cat_cols=cat_cols,
        cont_cols=cont_cols_for_pretrain,      # 타겟이 제거된 컬럼 리스트
        pretrain_epochs=pretrain_epochs
    )

    results = {}

    # 4. 그룹별 미세조정(Fine-tuning) 수행 (이하 동일)
    for group_name, group_data in finetune_groups.items():
        print(f"\n--- {group_name} 그룹 미세조정 시작 ---")

        model_to_finetune = copy.deepcopy(base_model)

        finetuned_model, y_scaler = finetune_saint_model(
            model=model_to_finetune,
            preprocessor=preprocessor,
            finetune_df=group_data['df'],
            target_col=target_col,
            finetune_epochs=finetune_epochs
        )

        results[group_name] = {
            "model": finetuned_model,
            "preprocessor": preprocessor,
            "y_scaler": y_scaler
        }

    print("\n\n===== 🚀 모든 그룹에 대한 학습 최종 완료 =====")
    return results

In [ ]:
pretrain_dfs = [train_group_1, train_group_2, train_group_3]

finetune_groups_dict = {
    "group1": {"df": train_group_1, "cont_cols": g1_cont_cols},
    "group2": {"df": train_group_2, "cont_cols": g2_cont_cols},
    "group3": {"df": train_group_3, "cont_cols": g3_cont_cols}
}

# # --- 함수 호출 ---
# 이제 target_col에 올바른 문자열 값이 전달됩니다.
training_results = run_full_training_pipeline(
    pretrain_dfs=pretrain_dfs,
    finetune_groups=finetune_groups_dict,
    target_col=TARGET_COL_LOG,
    cat_cols=CAT_COLS,
    common_cont_cols=g1_cont_cols, # 자가학습 기준이 되는 공통 피처
    pretrain_epochs=15,
    finetune_epochs=100
)

===== 🚀 1. SAINT 자기지도학습 시작 =====


✅ 자기지도학습 완료

--- group1 그룹 미세조정 시작 ---
===== 🚀 2. 전력소비량(kWh)_log에 대한 미세조정 시작 =====


✅ 미세조정 완료

--- group2 그룹 미세조정 시작 ---
===== 🚀 2. 전력소비량(kWh)_log에 대한 미세조정 시작 =====


✅ 미세조정 완료


===== 🚀 모든 그룹에 대한 학습 최종 완료 =====


In [ ]:
# 저장할 디렉토리 생성
save_dir = 'trained_models_final'
os.makedirs(save_dir, exist_ok=True)

for group_name, results in training_results.items():
    print(f"--- [{group_name}] 객체 저장 시작 ---")

    # 1. 모델(Model) 저장
    model_path = os.path.join(save_dir, f'model_{group_name}.pt')
    torch.save(results['model'].state_dict(), model_path)
    print(f"  ✅ 모델 저장 완료: {model_path}")

    # 2. 전처리기(Preprocessor) 저장
    # 해결: 'joblib.dump'를 사용하여 전처리기 객체를 직접 저장합니다.
    preprocessor_path = os.path.join(save_dir, f'preprocessor_{group_name}.pkl')
    joblib.dump(results['preprocessor'], preprocessor_path)
    print(f"  ✅ 전처리기 저장 완료: {preprocessor_path}")

    # 3. 스케일러(Scaler) 저장
    scaler_path = os.path.join(save_dir, f'y_scaler_{group_name}.pkl')
    joblib.dump(results['y_scaler'], scaler_path)
    print(f"  ✅ 스케일러 저장 완료: {scaler_path}")

--- [group1] 객체 저장 시작 ---
  ✅ 모델 저장 완료: trained_models_final/model_group1.pt
  ✅ 전처리기 저장 완료: trained_models_final/preprocessor_group1.pkl
  ✅ 스케일러 저장 완료: trained_models_final/y_scaler_group1.pkl
--- [group2] 객체 저장 시작 ---
  ✅ 모델 저장 완료: trained_models_final/model_group2.pt
  ✅ 전처리기 저장 완료: trained_models_final/preprocessor_group2.pkl
  ✅ 스케일러 저장 완료: trained_models_final/y_scaler_group2.pkl
--- [group3] 객체 저장 시작 ---
  ✅ 모델 저장 완료: trained_models_final/model_group3.pt
  ✅ 전처리기 저장 완료: trained_models_final/preprocessor_group3.pkl
  ✅ 스케일러 저장 완료: trained_models_final/y_scaler_group3.pkl


In [ ]:
import torch
import joblib
import os
from pytorch_widedeep.models import WideDeep, SAINT

# ---------------------------------------------------------------------------------
# ⚙️ 1. 모델 뼈대를 생성하는 최종 함수
# ---------------------------------------------------------------------------------
def get_model_for_group(group_name, preprocessor):
    """
    '불러온 preprocessor' 객체와 '에러 메시지'를 근거로
    모델 뼈대를 100% 동일하게 재구성합니다.
    """
    saint_params = {
        'n_blocks': 2,
        'n_heads': 8,
        'attn_dropout': 0.1,
        'ff_dropout': 0.1,
    }

    deeptabular = SAINT(
        column_idx=preprocessor.column_idx,
        cat_embed_input=preprocessor.cat_embed_input,
        continuous_cols=list(preprocessor.continuous_cols), # 리스트 형태로 변환
        **saint_params
    )

    model = WideDeep(deeptabular=deeptabular)

    return model

# ---------------------------------------------------------------------------------
# ⚙️ 2. 전체 불러오기 파이프라인
# ---------------------------------------------------------------------------------

# 저장된 파일들이 있는 디렉토리
load_dir = 'trained_models_final'

# 불러온 객체들을 저장할 딕셔너리
loaded_training_results = {}

# 불러올 그룹 이름 리스트
group_names = ['group1', 'group2', 'group3']

print("===== 🚀 저장된 객체 불러오기 시작 =====")

for group_name in group_names:
    print(f"\n--- [{group_name}] 객체 불러오기 시작 ---")

    # 1. 이 부분이 실행되고 있는지 확인해주세요.
    # 전처리기를 불러오는 코드 라인입니다.
    preprocessor_path = os.path.join(load_dir, f'preprocessor_{group_name}.pkl')
    preprocessor = joblib.load(preprocessor_path)

    # 2. 이 print문이 있는지 확인해주세요.
    # 불러오기 완료를 알려주는 출력 라인입니다.
    print(f"  ✅ 전처리기 불러오기 완료: {preprocessor_path}")

    # 모델 뼈대 생성
    model = get_model_for_group(group_name, preprocessor)

    # 모델 가중치 불러오기
    model_path = os.path.join(load_dir, f'model_{group_name}.pt')
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    print(f"  ✅ 모델 불러오기 완료: {model_path}")

    # 스케일러 불러오기
    scaler_path = os.path.join(load_dir, f'y_scaler_{group_name}.pkl')
    y_scaler = joblib.load(scaler_path)
    print(f"  ✅ 스케일러 불러오기 완료: {scaler_path}")

    # 불러온 객체들을 딕셔너리에 저장
    loaded_training_results[group_name] = {
        'model': model,
        'preprocessor': preprocessor,
        'y_scaler': y_scaler
    }

print("\n\n===== ✅ 모든 객체 불러오기 완료 =====")

===== 🚀 저장된 객체 불러오기 시작 =====

--- [group1] 객체 불러오기 시작 ---
  ✅ 전처리기 불러오기 완료: trained_models_final/preprocessor_group1.pkl
  ✅ 모델 불러오기 완료: trained_models_final/model_group1.pt
  ✅ 스케일러 불러오기 완료: trained_models_final/y_scaler_group1.pkl

--- [group2] 객체 불러오기 시작 ---
  ✅ 전처리기 불러오기 완료: trained_models_final/preprocessor_group2.pkl
  ✅ 모델 불러오기 완료: trained_models_final/model_group2.pt
  ✅ 스케일러 불러오기 완료: trained_models_final/y_scaler_group2.pkl

--- [group3] 객체 불러오기 시작 ---
  ✅ 전처리기 불러오기 완료: trained_models_final/preprocessor_group3.pkl
  ✅ 모델 불러오기 완료: trained_models_final/model_group3.pt
  ✅ 스케일러 불러오기 완료: trained_models_final/y_scaler_group3.pkl


===== ✅ 모든 객체 불러오기 완료 =====


In [ ]:
# 1. SMAPE 평가 함수 (이전에 정의한 함수)
# ======================================================================================
def smape(y_true, y_pred):
    """
    SMAPE (Symmetric Mean Absolute Percentage Error)를 계산하는 함수입니다.
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# ======================================================================================
# 2. 그룹별 예측 및 평가 실행
# ======================================================================================

# --- 테스트 데이터 준비 ---
# group_1_test_final, group_2_test_final, group_3_test_final이 정의되어 있다고 가정합니다.
test_groups = {
    "group1": valid_group_1,
    "group2": valid_group_2,
    "group3": valid_group_3,
}

# --- 최종 결과를 저장할 딕셔너리 ---
smape_scores = {}

print("===== 🚀 그룹별 테스트 데이터 예측 및 평가 시작 =====")

# training_results 딕셔너리를 순회하며 각 그룹에 대한 예측을 수행합니다.
for group_name, results in loaded_training_results.items():
    print(f"\n--- [{group_name}] 예측 평가 시작 ---")

    # 1. 해당 그룹의 학습된 객체들 가져오기
    model = results['model']
    preprocessor = results['preprocessor']
    y_scaler = results['y_scaler']

    # 2. 해당 그룹의 테스트 데이터 가져오기
    test_df = test_groups[group_name]

    # 3. 데이터 준비
    # X_test는 전처리를 위해 타겟 컬럼을 제외한 모든 컬럼을 포함할 수 있습니다.
    X_test = test_df
    # y_true는 로그 변환되기 전의 원본 '전력소비량(kWh)' 컬럼입니다.
    y_true = test_df['전력소비량(kWh)'].values

    # 4. 피처 전처리
    X_test_transformed = preprocessor.transform(X_test).astype('float32')

    # 5. 예측 수행 (메모리 효율적인 배치 처리)
    predictor = Trainer(model=model, objective='regression')
    batch_size = 1024
    n_batches = math.ceil(len(X_test_transformed) / batch_size)

    predictions_log_list = [] # 로그 스케일의 예측 결과를 저장할 리스트

    for i in range(n_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        X_batch = X_test_transformed[start:end]

        # 5-1. 모델 예측 (결과는 scaled log 값)
        y_pred_scaled_batch = predictor.predict(X_tab=X_batch, batch_size=batch_size)

        # 5-2. StandardScaler 역변환 (결과는 log 값)
        y_pred_log_batch = y_scaler.inverse_transform(y_pred_scaled_batch.reshape(-1, 1))

        predictions_log_list.append(y_pred_log_batch)

    # 6. 전체 예측 결과 취합 및 최종 역변환
    # 6-1. 모든 배치의 예측 결과를 합칩니다 (결과는 아직 log 스케일).
    y_pred_log = np.vstack(predictions_log_list)

    # 6-2. 로그 역변환(expm1)을 통해 최종적으로 원래 스케일로 복원합니다.
    y_pred_original = np.expm1(y_pred_log)

    # 7. SMAPE 계산 및 저장
    score = smape(y_true, y_pred_original.flatten())
    smape_scores[group_name] = score

    print(f"✅ [{group_name}] 예측 완료! SMAPE: {score:.4f} %")

print("\n\n===== ✅ 모든 그룹 평가 완료 =====")
for group_name, score in smape_scores.items():
    print(f"   - {group_name} SMAPE: {score:.4f} %")

===== 🚀 그룹별 테스트 데이터 예측 및 평가 시작 =====

--- [group1] 예측 평가 시작 ---


✅ [group3] 예측 완료! SMAPE: 10.7259 %


===== ✅ 모든 그룹 평가 완료 =====
   - group1 SMAPE: 13.5345 %
   - group2 SMAPE: 9.9692 %
   - group3 SMAPE: 10.7259 %


In [ ]:
def predict_for_single_group(group_name, loaded_training_results, validation_groups, test_groups):
    """
    지정된 단일 그룹에 대한 재귀 예측을 수행합니다.
    (시차 피처 생성 로직 수정 완료)
    """
    print(f"\n--- [{group_name}] 예측 시작 ---")

    # ------------------- 1. 객체 및 데이터 준비 -------------------
    results = loaded_training_results[group_name]
    model = results['model']
    preprocessor = results['preprocessor']
    y_scaler = results['y_scaler']

    predictor = Trainer(model=model, objective='regression')
    PREDICTION_HORIZON = len(test_groups[group_name])
    print(f"   - 예측 기간(스텝 수): {PREDICTION_HORIZON}")

    initial_window_df = validation_groups[group_name].tail(168).copy()
    recursive_df = initial_window_df.copy()
    future_rows_list = []

    # ------------------- 2. 한 스텝씩 재귀 예측 수행 -------------------
    for i in range(PREDICTION_HORIZON):
        print(f"   - {i+1}번째 스텝 예측 중...")
        # 2-1. 현재 스텝의 예측 입력(X) 준비
        current_step_df = recursive_df.tail(1)
        X_pred = preprocessor.transform(current_step_df).astype('float32')

        # 2-2. 1개 스텝 예측
        y_pred_scaled = predictor.predict(X_tab=X_pred, batch_size=32)

        # 2-3. 예측값 역변환
        y_pred_log = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))
        y_pred_original = np.expm1(y_pred_log).flatten()[0]

        # ✅ NaN 값 발생 시 예측 중단
        if pd.isna(y_pred_original):
            print(f"스텝 {i+1}에서 NaN이 발생하여 [{group_name}] 그룹의 예측을 중단합니다! ❗️❗️❗️")
            break

        # 2-4. 다음 스텝을 위한 피처 업데이트
        next_step_info = test_groups[group_name].iloc[[i]].copy()
        next_step_info['전력소비량(kWh)'] = y_pred_original
        next_step_info['전력소비량(kWh)_log'] = y_pred_log.flatten()[0]

        next_step_info['1시간_전_전력소비량'] = recursive_df['전력소비량(kWh)'].iloc[-1]
        next_step_info['하루_전_전력소비량'] = recursive_df['전력소비량(kWh)'].iloc[-24]
        next_step_info['일주일_전_전력소비량'] = recursive_df['전력소비량(kWh)'].iloc[-168]

        future_rows_list.append(next_step_info)
        recursive_df = pd.concat([recursive_df, next_step_info], ignore_index=True)

    # ------------------- 3. 최종 결과 취합 -------------------
    if not future_rows_list:
        print(f"   - [{group_name}] 그룹에 대한 예측값이 없습니다.")
        return pd.DataFrame()

    predictions_df = pd.concat(future_rows_list, ignore_index=True)

    temp_answer_df = pd.DataFrame({
        'num_date_time': predictions_df['num_date_time'],
        '전력소비량(kWh)': predictions_df['전력소비량(kWh)']
    })

    print(f"✅ [{group_name}] 예측 완료! {len(temp_answer_df)}개의 예측값 생성.")
    return temp_answer_df

In [ ]:
validation_groups = {
    "group1": valid_group_1,
    "group2": valid_group_2,
    "group3": valid_group_3,
}
test_groups = {
    "group1": test_group_1,
    "group2": test_group_2,
    "group3": test_group_3,
}

all_predictions_list = []

# --- 'group1'에 대해서만 예측 실행 ---
print("\n===== 🚀 단일 그룹('group1') 예측 테스트 시작 =====")
group_1_predictions = predict_for_single_group(
    group_name='group1',
    loaded_training_results=loaded_training_results,
    validation_groups=validation_groups,
    test_groups=test_groups
)
# all_predictions_list.append(group1_predictions)

In [ ]:
print("\n===== 🚀 단일 그룹('group2') 예측 테스트 시작 =====")
group_2_predictions = predict_for_single_group(
    group_name='group2',
    loaded_training_results=loaded_training_results,
    validation_groups=validation_groups,
    test_groups=test_groups
)

In [ ]:
print("\n===== 🚀 단일 그룹('group3') 예측 테스트 시작 =====")
group_3_predictions = predict_for_single_group(
    group_name='group3',
    loaded_training_results=loaded_training_results,
    validation_groups=validation_groups,
    test_groups=test_groups
)

In [ ]:
group_1_predictions.to_csv('group_1_prediction.csv', index = False)

In [ ]:
group_2_predictions.to_csv('group_2_prediction.csv', index = False)

In [ ]:
group_3_predictions.to_csv('group_3_prediction.csv', index = False)

In [ ]:
group_1_answer = pd.read_csv('group_1_prediction.csv')
group_2_answer = pd.read_csv('group_2_prediction.csv')
group_3_answer = pd.read_csv('group_3_prediction.csv')

In [ ]:
answer = pd.concat([group_1_answer, group_2_answer, group_3_answer], ignore_index=True)
answer.head()

,num_date_time,전력소비량(kWh)
0,1_20240825 00,6495.1255
1,1_20240825 01,2478.0930
2,1_20240825 02,3330.1997
3,1_20240825 03,1462.9817
4,1_20240825 04,1651.8745


In [ ]:
answer.to_csv('final_answer.csv', index = False)